In [0]:
# Adapted from
# https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [195]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [196]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.9.0-rc0


### Step 1: Generate sample equations

In [0]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [0]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [199]:
# Parameters for the model and dataset.
TRAINING_SIZE = 1000
DIGITS = 3
# REVERSE = True
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789- '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}-{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a - b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 1000


In [200]:
questions[0]

'49-63  '

In [201]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)


Vectorization...


In [202]:
len(x[0])

7

In [203]:
len(questions[0])

7

In [204]:
questions[0]

'49-63  '

In [205]:
x[0]

array([[False, False, False, False, False, False,  True, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False,  True,
        False, False, False],
       [False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [206]:
y[0]

array([[False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [207]:
expected[0]

'-14 '

In [0]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

### Step 2: Training/Validation Split

In [209]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(900, 7, 12)
(900, 4, 12)
Validation Data:
(100, 7, 12)
(100, 4, 12)


### Step 3: Create Model

In [210]:
# input shape: 7 digits, each being 0-9, + or space (12 possibilities)
MAXLEN, len(chars)

(7, 12)

In [211]:
from keras.models import Sequential
from keras import layers

# Try replacing LSTM, GRU, or SimpleRNN.
RNN = layers.LSTM
# RNN = layers.SimpleRNN
# RNN = layers.GRU
HIDDEN_SIZE = 128
BATCH_SIZE = 128

print('Build model...')
model = Sequential()
# encoder: simply encode the same input 4 times 
model.add(RNN(units=HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))

# decoder: have 4 temporal outputs one for each of the digits of the results
# return_sequences=True tells it to keep all 4 temporal outputs, not only the final one (we need all four digits for the results)
model.add(RNN(units=HIDDEN_SIZE, return_sequences=True))

model.add(layers.Dense(name='classifier', units=len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_9 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
classifier (Dense)           (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [212]:
model.predict(np.array([x_val[0]]))

array([[[0.08386109, 0.08370347, 0.08297401, 0.0837782 , 0.08257446,
         0.08340053, 0.08413669, 0.08384574, 0.08198585, 0.08306871,
         0.08307053, 0.08360069],
        [0.08417597, 0.08401353, 0.08272962, 0.08408671, 0.08208953,
         0.08335014, 0.08466751, 0.08416119, 0.08115227, 0.0828871 ,
         0.08285623, 0.08383021],
        [0.08434068, 0.08426883, 0.08255812, 0.08429851, 0.0817899 ,
         0.0832445 , 0.08502401, 0.08435749, 0.08065177, 0.08276303,
         0.08268684, 0.08401636],
        [0.08440474, 0.08447564, 0.08243638, 0.08444344, 0.08161285,
         0.08311924, 0.08526792, 0.08448039, 0.08036436, 0.08267817,
         0.08255589, 0.08416098]]], dtype=float32)

In [213]:
model.predict_classes(np.array([x_val[0]]))

array([[6, 6, 6, 6]])

### Step 4: Train

In [214]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 20):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 3s 3ms/step - loss: 2.4388 - acc: 0.2342 - val_loss: 2.3464 - val_acc: 0.2550
Q 58-752  T -694 ☒     
Q 5-931   T -926 ☒     
Q 213-4   T 209  ☒     
Q 263-102 T 161  ☒     
Q 3-392   T -389 ☒     
Q 8-215   T -207 ☒     
Q 29-5    T 24   ☒     
Q 509-95  T 414  ☒     
Q 6-59    T -53  ☒     
Q 9-44    T -35  ☒     

--------------------------------------------------
Iteration 2
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 463us/step - loss: 2.2549 - acc: 0.2603 - val_loss: 2.2222 - val_acc: 0.2550
Q 44-531  T -487 ☒     
Q 623-2   T 621  ☒     
Q 44-531  T -487 ☒     
Q 58-155  T -97  ☒     
Q 804-86  T 718  ☒     
Q 36-2    T 34   ☒     
Q 61-187  T -126 ☒     
Q 5-931   T -926 ☒     
Q 775-74  T 701  ☒     
Q 2-5     T -3   ☒     

-----------------------------------

900/900 [==============================] - 0s 470us/step - loss: 2.0962 - acc: 0.2700 - val_loss: 2.1095 - val_acc: 0.2725
Q 34-78   T -44  ☒     
Q 6-75    T -69  ☒     
Q 29-5    T 24   ☒     
Q 623-2   T 621  ☒     
Q 9-44    T -35  ☒     
Q 804-86  T 718  ☒     
Q 21-343  T -322 ☒     
Q 599-5   T 594  ☒     
Q 47-23   T 24   ☒     
Q 55-3    T 52   ☒     

--------------------------------------------------
Iteration 8
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 469us/step - loss: 2.0816 - acc: 0.2872 - val_loss: 2.0922 - val_acc: 0.3275
Q 4-83    T -79  ☒     
Q 0-41    T -41  ☒     
Q 263-102 T 161  ☒ -   
Q 53-26   T 27   ☒ -   
Q 58-155  T -97  ☒ -   
Q 4-82    T -78  ☒     
Q 47-23   T 24   ☒ -   
Q 58-752  T -694 ☒ -   
Q 1-182   T -181 ☒ -   
Q 770-364 T 406  ☒ -   

--------------------------------------------------
Iteration 9
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [=========================

900/900 [==============================] - 0s 454us/step - loss: 1.9710 - acc: 0.3847 - val_loss: 1.9013 - val_acc: 0.4000
Q 61-187  T -126 ☒ --  
Q 401-611 T -210 ☒ --  
Q 401-611 T -210 ☒ --  
Q 804-86  T 718  ☒ --  
Q 0-803   T -803 ☒ --  
Q 5-17    T -12  ☒ -1  
Q 8-215   T -207 ☒ --  
Q 61-628  T -567 ☒ --  
Q 770-364 T 406  ☒ --3 
Q 454-7   T 447  ☒ --  

--------------------------------------------------
Iteration 14
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [==============================] - 0s 462us/step - loss: 1.9107 - acc: 0.3950 - val_loss: 1.8377 - val_acc: 0.4150
Q 61-628  T -567 ☒ -33 
Q 388-90  T 298  ☒ -3  
Q 241-2   T 239  ☒ -3  
Q 29-40   T -11  ☒ -3  
Q 1-8     T -7   ☒ -   
Q 61-628  T -567 ☒ -33 
Q 148-9   T 139  ☒ -3  
Q 95-8    T 87   ☒ -3  
Q 65-784  T -719 ☒ -33 
Q 61-187  T -126 ☒ -33 

--------------------------------------------------
Iteration 15
Train on 900 samples, validate on 100 samples
Epoch 1/1
900/900 [=======================

900/900 [==============================] - 0s 460us/step - loss: 1.8312 - acc: 0.3992 - val_loss: 1.8157 - val_acc: 0.4300
Q 335-858 T -523 ☒ -3  
Q 316-368 T -52  ☒ -31 
Q 263-102 T 161  ☒ -1  
Q 401-611 T -210 ☒ -11 
Q 62-61   T 1    ☒ -1  
Q 6-75    T -69  ☒ -1  
Q 58-155  T -97  ☒ -1  
Q 905-9   T 896  ☒ -5  
Q 595-70  T 525  ☒ -1  
Q 94-508  T -414 ☒ -11 


In [215]:
x_val[0]

array([[False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
         True, False, False],
       [False, False, False,  True, False, False, False, False, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False,  True, False, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [216]:
model.predict(np.array([x_val[0]]))

array([[[1.58960316e-02, 4.44127530e-01, 1.56778097e-02, 7.75579959e-02,
         7.04842433e-02, 5.12277521e-02, 6.72021508e-02, 5.98344468e-02,
         4.41482402e-02, 4.84528989e-02, 6.41117245e-02, 4.12791930e-02],
        [5.50463200e-02, 1.05762237e-03, 6.15860038e-02, 1.16197370e-01,
         9.96844396e-02, 1.02129504e-01, 1.05371594e-01, 1.05011173e-01,
         9.63835493e-02, 1.00266188e-01, 8.21479186e-02, 7.51183853e-02],
        [5.04306972e-01, 6.08452729e-06, 4.91322838e-02, 8.65036324e-02,
         4.12809215e-02, 3.83484215e-02, 6.44010082e-02, 4.91278917e-02,
         4.97538410e-02, 4.00490612e-02, 4.09311689e-02, 3.61587368e-02],
        [9.93111372e-01, 2.72310507e-09, 7.87661120e-04, 1.26818428e-03,
         5.91886404e-04, 3.92614980e-04, 7.54354696e-04, 7.27016595e-04,
         6.33666583e-04, 4.58022521e-04, 6.06012356e-04, 6.69285888e-04]]],
      dtype=float32)

In [217]:
model.predict_classes(np.array([x_val[0]]))

array([[1, 3, 0, 0]])